# Data Aggregation and Group Operations

In [ ]:
import pandas as pd
import numpy as np

## GroupBy mechanics

In [ ]:
nrows = 10

df = pd.DataFrame({'company' : np.random.choice(list('ab'), nrows),
                   'data1' : np.random.randn(nrows) * 50 + 100,
                   'city' : np.random.choice(list('MP'), nrows),
                   'income' : np.random.randn(nrows) * 30000 + 50000 })

df

#### Ejercicio: 
Añade una columna en el df con el total_income por ciudad

In [ ]:
df.groupby('city').sum()['income'].reset_index() #al hacer esto añado una columna

In [ ]:
sum_income = df.groupby('city').sum()['income'].reset_index()

In [ ]:
df.merge(sum_income,  left_on = 'city', right_on = 'city', suffixes =['', '_total_city']).sort_index(axis=1)

In [ ]:
mean_income_city_company = df.groupby(['city','company']).mean()['income'].reset_index()

In [ ]:
mean_income_city_company

In [ ]:
df = df.merge(mean_income_city_company, left_on = ['city', 'company'], right_on = ['city','company'], suffixes = ['', '_mean'])

In [ ]:
df

In [ ]:
df['over_mean'] = df['income']>df['income_mean']

In [ ]:
df

In [ ]:
grouped = df.groupby('company') #hasta que no diga la operacion de agregacion no hace nada
grouped

In [ ]:
grouped.sum()

In [ ]:
grouped.sum()['income'] #me da una serie

In [ ]:
df.groupby('company')['income'].sum()

In [ ]:
grouped.mean()

In [ ]:
df1 = grouped.mean()

In [ ]:
df1.index #por defecto el group by devuelve un dataframe donde el indice es el campo por el que agrupo

In [ ]:
df1.reset_index() #así quito el campo de agrupación del índice

In [ ]:
df.groupby(['company', 'city']).mean()

In [ ]:
df.groupby(['company', 'city']).mean()['income']

In [ ]:
df.groupby(['company', 'city'])['income'].mean()

In [ ]:
means = df.groupby(['company', 'city'])['income'].mean()

In [ ]:
means

In [ ]:
means.index

In [ ]:
means['a', 'M']

In [ ]:
means.reset_index()

### Iterating over groups

In [ ]:
for name, group in df.groupby('company'):
    print(group)

In [ ]:
all_data = dict(list(df.groupby('company')))
all_data['a']

### Selecting a column or subset of columns

In [ ]:
df

In [ ]:
df.groupby('company')[['data1', 'city']].max().reset_index()

In [ ]:
df[df['company']=='a']

In [ ]:
df[df['company']=='a'][['data1', 'city']].max().reset_index()

In [ ]:
df[df['company']=='b'][['data1', 'city']].max()

## Data aggregation

In [ ]:
df.groupby('city').median()

In [ ]:
df.groupby('city').quantile(.9)

In [ ]:
stats = df.groupby('city').describe()
stats

In [ ]:
stats['data1','mean']

In [ ]:
stats.columns

In [ ]:
!wget https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv

In [ ]:
tips = pd.read_csv('tips.csv')
tips.head()

In [ ]:
tips.time.unique()

In [ ]:
tips.day.unique()

In [ ]:
tips.describe()

In [ ]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

In [ ]:
l_stats=['min','max','mean', 'std', 'median', np.sum]

In [ ]:
tips.groupby('time')[['total_bill', 'tip', 'tip_pct', 'size']].agg(l_stats).T

### Column-wise and multiple function application

In [ ]:
tips.groupby('sex').mean()

In [ ]:
tips.groupby('sex').std()

In [ ]:
tips.groupby('sex').agg(['mean', 'std'])

In [ ]:
tips.groupby(['sex', 'smoker']).agg(['mean', 'std', 'count'])

In [ ]:
tips.groupby(['sex', 'smoker']).agg([np.mean, np.std, np.count_nonzero])

In [ ]:
def minimax(series):
    return series.max() - series.min()

tips.groupby(['sex', 'smoker']).agg([np.mean, np.std, minimax])

In [ ]:
tips.groupby(['sex', 'smoker']).agg({'total_bill' : [np.sum, np.mean], 
                                     'tip' : [minimax, np.std]})

## Group-wise operations and transformations

In [ ]:
tips_by_smoker = tips.groupby('smoker')['tip_pct'].agg([np.mean, np.std])
tips_by_smoker

In [ ]:
merged = tips.merge(tips_by_smoker, left_on='smoker', right_index=True)
merged.head()

### Apply: General split-apply-combine

In [ ]:
def top(df, n=5, col='tip_pct'):
    return df.sort_values(by=col, ascending=False).head(n)

top(merged)

In [ ]:
merged.groupby('sex').apply(top)

#### Suppressing the group keys

In [ ]:
tips.groupby(['sex', 'smoker'], group_keys=False).apply(top)

### Quantile and bucket analysis

In [ ]:
pd.cut(merged['total_bill'], 5)

In [ ]:
merged.groupby(pd.cut(merged['total_bill'], 5))['tip_pct'].agg([np.mean, np.std])

In [ ]:
merged.groupby(merged['size'] > 2)['tip_pct'].agg([np.mean, np.std])

In [ ]:
merged.groupby(merged['size'] == 2)['tip_pct'].agg([np.mean, np.std])